https://www.kaggle.com/competitions/spaceship-titanic/overview<br>To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly using records recovered from the spaceship’s damaged computer system.
<br>
Help save them and change history!

In [161]:
#disable some annoying warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor


In [135]:
dataset = pd.read_csv("train.csv")
test_set = pd.read_csv("test.csv")
print(dataset.shape, dataset.columns)
dataset.head()


(8693, 14) Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [136]:
test_set.shape

(4277, 13)

In [138]:
#dataset.drop(columns=['HomePlanet', 'Cabin', 'Destination','Name'], inplace=True)
test_set.drop(columns=['HomePlanet', 'Cabin', 'Destination','Name'], inplace=True)
test_set.shape

(4277, 9)

In [92]:
# Calculate the correlation matrix
#corr_matrix = dataset.corr()

# Print the correlation between the 14th dimension and all other columns
#corr_last = corr_matrix.iloc[-1, :]
#print(corr_last)
# Plot the correlation matrix
#sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
#plt.show()

In [93]:
#drop inf and NaN
data = dataset.dropna()
data = data[~data.isin([np.nan, np.inf, -np.inf]).any(1)]
data.head()

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0001_01,False,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,0002_01,False,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,0003_01,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,0003_02,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,0004_01,False,16.0,False,303.0,70.0,151.0,565.0,2.0,True


In [140]:

test_set = test_set.fillna(test_set.mean())
test_set = test_set[~test_set.isin([np.nan, np.inf, -np.inf]).any(1)]
test_set.shape

(4277, 9)

In [141]:
# Divide the dataframe into features (X) and target (y)
X = data.iloc[:, :9]
y = data.iloc[:, 9]

In [142]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [143]:
# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

RandomForestClassifier()

In [144]:
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.7615437629221227


Let's try on the whole dataset

In [145]:
X_train = data.iloc[:, :9]
y_train = data.iloc[:, 9]
X_test = test_set

In [174]:
#X_test.head()

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [159]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [162]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [163]:
rf_random.best_params_

{'n_estimators': 1600,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

In [165]:
clf = RandomForestClassifier(n_estimators=1600, min_samples_split=2, min_samples_leaf=4, max_features = "sqrt", max_depth=10, bootstrap=True )
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_leaf=4,
                       n_estimators=1600)

In [167]:
y_test = clf.predict(X_test)
len(y_test) #should be 4277

4277

In [168]:
passenger_ids = X_test['PassengerId']

In [169]:
df = pd.DataFrame({'Transported': y_test})
df.shape

(4277, 1)

In [170]:
final_df = pd.concat([passenger_ids, df], axis=1)

In [171]:
final_df.to_csv('prediction2.csv', index=False)

# Score: 0.76151 - 1st try - Random Forest


1842 out of 2351

# SCORE - 0.78793 2nd try - Random Forest Grid Search

1501 out of 2351

# Convert all categorical data into numerical
https://www.kdnuggets.com/2021/05/deal-with-categorical-data-machine-learning.html

### HomePlanet

In [3]:
#check how many types
data["HomePlanet"].drop_duplicates()

0     Europa
1      Earth
12      Mars
59       NaN
Name: HomePlanet, dtype: object

In [4]:
#cross table overview
pd.crosstab(data.HomePlanet, data.Transported)

Transported,False,True
HomePlanet,,
Earth,2651,1951
Europa,727,1404
Mars,839,920


Huge gap between Earth and Europa native. Not sure if it is worth to keep Martians, lol<br>
ok let's assign numerical values<br><b>
Earth = 1<br>
Europa = 2<br>
Mars = 3</b>

In [5]:
#ok let's assign numerical values
#Planet_Num = {'Earth': 1,'Europa': 2,'Mars': 3}
#data["HomePlanet"] = data.HomePlanet.map(Planet_Num)

#fill NaN with most frequent values = df.mode()
#data["HomePlanet"].fillna(value=data["HomePlanet"].mode().iloc[0], inplace=True)

#data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,2.0,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,1.0,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,2.0,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,2.0,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,1.0,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


### CryoSleep

In [147]:
print(data["CryoSleep"].drop_duplicates())
#check how many Nan
print("N of Nan values in CRYOSLEEP:", data["CryoSleep"].isna().sum())
#cross table overview
pd.crosstab(data.CryoSleep, data.Transported)


0     False
7      True
92      NaN
Name: CryoSleep, dtype: object
N of Nan values in CRYOSLEEP: 217


Transported,False,True
CryoSleep,,
False,3650,1789
True,554,2483


In [148]:
#fill NaN with most frequent values = df.mode()----->TO OPTIMIZE TRAIN A LOGISTIC REGRESSOR TO PREDICT NAN INSTEAD!!!
data["CryoSleep"].fillna(value=data["CryoSleep"].mode().iloc[0], inplace=True)

### Cabin
Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.<br>
<b>Idea</b>: Divide by deck/num/side and check for correlation with transported
<b> Implementation </b> Copy values which ends with P to a new column, same for S and compare with Transported<BR> OR <BR> divide and create many columns by alphabetical order e.g. from A to D, from D to G etc.etc. and compare w/ Transported.

In [8]:
#check how many Nan
print("N of Nan values in Cabin:", data["Cabin"].isna().sum())
#fill in with some random string
#data["Cabin"].fillna(value= "boh", inplace=True) #yay



N of Nan values in Cabin: 199
